In [ ]:


import os
groq_api_key = os.getenv("GROQ_API_KEY", "gsk_...")

In [ ]:
from langchain_groq import ChatGroq
from pydantic import BaseModel
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import  ChatPromptTemplate

In [ ]:
llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=4000,
    timeout=None,
    api_key=groq_api_key,
)


In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
import os
import tiktoken

# file_path = "assets/serial Port persianmikrotik.ir.pdf"
# loader = PyMuPDFLoader(file_path)
# docs=loader.load()


pdf_path=[]
for root, dirs, files in os.walk("./assets"):
    for file in files:
        if file.endswith(".pdf"):
            pdf_path.append(os.path.join(root, file))


docs=[]
for pdf in pdf_path:
    loader = PyMuPDFLoader(pdf)
    docs.extend(loader.load())


def format_text(documents):
    return '\n\n'.join([p.page_content for p in documents])


context= format_text(docs)


In [ ]:
import tiktoken
encoding=tiktoken.encoding_for_model("gpt-4o-mini")

In [ ]:


from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (SystemMessagePromptTemplate, HumanMessagePromptTemplate,
                                    ChatPromptTemplate)

system = SystemMessagePromptTemplate.from_template("""You are helpful AI assistant who works as document summarizer.
                                                   You must not hallucinate or provide any false information.""")

prompt = """Summarize the given context in {words}.
            ### Context:
            {context}

            ### Summary:"""

prompt = HumanMessagePromptTemplate.from_template(prompt)

messages = [system, prompt]
template = ChatPromptTemplate(messages)

summary_chain = template | llm | StrOutputParser()
summary_chain


In [ ]:
response = summary_chain.invoke({'context': context[:5000], 'words': 50})
print(response)